In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder.getOrCreate()




In [ ]:
!wget https://github.com/user-attachments/files/18538626/All.Trials.2201.2025.tar.gz

In [4]:
df = spark.read.parquet("all_trials_22012025")

df.show(5)

+-----------+----------+-----------+-------------------+---------------+--------------------+--------------+--------------+-----------+------------+
|     nct_id|   drug_id| disease_id|mapped_drug_disease|trial_in_chembl|      disease_labels|   drug_labels|    trial_type|trial_phase|trial_status|
+-----------+----------+-----------+-------------------+---------------+--------------------+--------------+--------------+-----------+------------+
|NCT00000553|      NULL|EFO_0000319|              false|           true|[cardiovascular d...|[antioxidants]|INTERVENTIONAL|     PHASE3|   COMPLETED|
|NCT00000553|      NULL|EFO_0001645|              false|           true|  [coronary disease]|[antioxidants]|INTERVENTIONAL|     PHASE3|   COMPLETED|
|NCT00000553|      NULL|EFO_0003777|              false|           true|    [heart diseases]|[antioxidants]|INTERVENTIONAL|     PHASE3|   COMPLETED|
|NCT00000553|      NULL|EFO_1001375|              false|           true|[myocardial ische...|[antioxidants

In [14]:
df.filter(f.col("trial_phase") == "UNKNOWN").show(10)

+-----------+------------+-------------+-------------------+---------------+--------------------+-------------------+--------------+-----------+------------+
|     nct_id|     drug_id|   disease_id|mapped_drug_disease|trial_in_chembl|      disease_labels|        drug_labels|    trial_type|trial_phase|trial_status|
+-----------+------------+-------------+-------------------+---------------+--------------------+-------------------+--------------+-----------+------------+
|NCT00104728|   CHEMBL939|MONDO_0021117|               true|          false|    [lung neoplasms]|        [gefitinib]|INTERVENTIONAL|    UNKNOWN|  TERMINATED|
|NCT00117585|        NULL|         NULL|              false|          false|[hypotension, ort...|[treatment phase 2]|INTERVENTIONAL|    UNKNOWN|   COMPLETED|
|NCT00117585|        NULL|  EFO_0005251|              false|          false|       [hypotension]|[treatment phase 2]|INTERVENTIONAL|    UNKNOWN|   COMPLETED|
|NCT00120042|CHEMBL395429|         NULL|            

In [6]:
df.filter(f.col("trial_phase") == "UNKNOWN").groupBy("mapped_drug_disease").count().show(10)

+-------------------+-----+
|mapped_drug_disease|count|
+-------------------+-----+
|               true|34924|
|              false|26879|
+-------------------+-----+



In [4]:
from sqlmodel import create_engine, select, Session, SQLModel
from pyspark.sql import DataFrame, SparkSession

def create_spark_session() -> SparkSession:
    return SparkSession.builder \
        .appName("AACT Analysis") \
        .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
        .getOrCreate()

def load_aact_table(table_name: str, db_url: str, user: str, password: str, schema: str = "ctgov", limit: int | None = None) -> DataFrame:
    """Load a table from AACT with optional row limit"""
    jdbc_url = f"jdbc:postgresql://{db_url}"
    
    connection_properties = {
        "user": user,
        "password": password,
        "driver": "org.postgresql.Driver",
        "ssl": "true",
        "sslmode": "require"
    }
    
    full_table_name = f"{schema}.{table_name}"
    
    if limit:
        query = f"(SELECT * FROM {full_table_name} LIMIT {limit}) as tmp"
        return SparkSession.builder.getOrCreate().read.jdbc(url=jdbc_url, table=query, properties=connection_properties)
    else:
        return SparkSession.builder.getOrCreate().read.jdbc(url=jdbc_url, table=full_table_name, properties=connection_properties)


In [6]:
# from .src.utils.db import create_spark_session, load_aact_table

spark = create_spark_session()

db_url = 'aact-db.ctti-clinicaltrials.org:5432/aact'
user = 'irenelopez'
password = 'Ephemeral2023'

studies = load_aact_table("studies", db_url, user, password, limit=1000)
studies.show(5)



25/04/24 10:52:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-----------------------------+--------------------------+----------------------------+--------------------------------+--------------------------+-----------------------------+-----------------------+----------------------------+-------------------------------+-------------------------+------------------------------+-----------------------------------+-----------------------------+----------------------------------+-----------------------------+-----------------------+----------------------------+----------------+---------------+----------+-----------------------+-----------------+---------------------+--------------------+---------------+-----------------------------+----------------------------+-----------------------+---------------+--------------+-------+-------------------+--------------------+--------------------+--------------+-----------------+-------------+----------+---------------+--------------------+-----------------------+--------------+----------------+-----

In [12]:
class AACTConnector:
    """A connection manager for AACT database with credential persistence"""

    def __init__(self, db_url: str, user: str, password: str, schema: str = "ctgov"):
        """
        Initialize the connection manager.

        Args:
            db_url: Database host and port (e.g., "aact-db.ctti-clinicaltrials.org:5432/aact")
            user: Database username
            password: Database password
            schema: Database schema (defaults to "ctgov")
        """
        self.jdbc_url = f"jdbc:postgresql://{db_url}"
        self.connection_properties = {
            "user": user,
            "password": password,
            "driver": "org.postgresql.Driver",
            "ssl": "true",
            "sslmode": "require",
        }
        self.schema = schema
        self.spark = self._create_spark_session()

    def _create_spark_session(self) -> SparkSession:
        """Create a Spark session with PostgreSQL driver"""
        return (
            SparkSession.builder.appName("AACT Analysis")
            .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0")
            .getOrCreate()
        )

    def load_table(
        self,
        table_name: str,
        limit: int | None = None,
        select_cols: list[str] | str | None = None,
    ) -> DataFrame:
        """
        Load a table from AACT database.

        Args:
            table_name: Name of the table to load
            limit: Optional row limit for testing

        Returns:
            A Spark DataFrame containing the table data
        """
        full_table_name = f"{self.schema}.{table_name}"

        if limit:
            if select_cols:
                select_cols = ", ".join(select_cols) if isinstance(select_cols, list) else select_cols
            query = f"(SELECT {select_cols} FROM {full_table_name} LIMIT {limit}) as tmp"
            return self.spark.read.jdbc(
                url=self.jdbc_url, table=query, properties=self.connection_properties
            )
        else:
            return self.spark.read.jdbc(
                url=self.jdbc_url,
                table=full_table_name,
                properties=self.connection_properties,
            )

    def print_table_schema(self, table_name: str) -> None:
        """Get schema for a table"""
        limited_df = self.load_table(table_name, limit=1)
        limited_df.printSchema()


In [13]:
aact = AACTConnector(
    db_url="aact-db.ctti-clinicaltrials.org:5432/aact",
    user="irenelopez",
    password="Ephemeral2023"
)

In [11]:
aact.print_table_schema('studies')

root
 |-- nct_id: string (nullable = true)
 |-- nlm_download_date_description: string (nullable = true)
 |-- study_first_submitted_date: date (nullable = true)
 |-- results_first_submitted_date: date (nullable = true)
 |-- disposition_first_submitted_date: date (nullable = true)
 |-- last_update_submitted_date: date (nullable = true)
 |-- study_first_submitted_qc_date: date (nullable = true)
 |-- study_first_posted_date: date (nullable = true)
 |-- study_first_posted_date_type: string (nullable = true)
 |-- results_first_submitted_qc_date: date (nullable = true)
 |-- results_first_posted_date: date (nullable = true)
 |-- results_first_posted_date_type: string (nullable = true)
 |-- disposition_first_submitted_qc_date: date (nullable = true)
 |-- disposition_first_posted_date: date (nullable = true)
 |-- disposition_first_posted_date_type: string (nullable = true)
 |-- last_update_submitted_qc_date: date (nullable = true)
 |-- last_update_posted_date: date (nullable = true)
 |-- last_up

In [8]:
studies = aact.load_table("studies")
interventions = aact.load_table("interventions", limit=1000)

studies.show()
interventions.count()


+-----------+-----------------------------+--------------------------+----------------------------+--------------------------------+--------------------------+-----------------------------+-----------------------+----------------------------+-------------------------------+-------------------------+------------------------------+-----------------------------------+-----------------------------+----------------------------------+-----------------------------+-----------------------+----------------------------+----------------+---------------+----------+-----------------------+-----------------+---------------------+--------------------+---------------+-----------------------------+----------------------------+-----------------------+---------------+--------------+-----------+--------------------+--------------------+--------------------+--------------------+------------------+------+----------+---------------+--------------------+-----------------------+--------------+----------------+

1000

In [15]:
aact.load_table("studies", limit=1000, select_cols=["nct_id"]).show()

+-----------+
|     nct_id|
+-----------+
|NCT00000102|
|NCT00000104|
|NCT00000105|
|NCT00000106|
|NCT00000107|
|NCT00000108|
|NCT00000110|
|NCT00000111|
|NCT00000112|
|NCT00000113|
|NCT00000114|
|NCT00000115|
|NCT00000116|
|NCT00000117|
|NCT00000118|
|NCT00000119|
|NCT00000120|
|NCT00000121|
|NCT00000122|
|NCT00000123|
+-----------+
only showing top 20 rows

